In [1]:
import sys
sys.path.insert(0, "/Users/artur/workspace/recipedb")

In [2]:
from recipedb.db import get_db
import functools

In [3]:
db = get_db()

In [4]:
exs = db.allrecipes.find({"error":{"$eq":None}})

In [5]:
NUM_EXAMPLES = 30000
exs_recipes = list(map(lambda x: next(exs), range(NUM_EXAMPLES))) # grab the next NUM_EXAMPLES from cursor above

In [6]:
def write_list_file(file, lst):
    with open(file, "w+") as f:
        f.writelines(map(lambda x: "%s\n"%x,list(lst)))

In [7]:
# Lets try simple counts first and then lets see if tf idf improves it, first step is finding common ingredients
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import pandas as pd

In [8]:
# add up all the list of list of ingredients into long corpus of ingredients
corpus = functools.reduce(lambda a,b: a+b, list(map(lambda x: x['data']['ingredients'], exs_recipes)))

In [9]:
vectorizer = CountVectorizer(analyzer='word',ngram_range=(1,5),)
tfidf = TfidfVectorizer(ngram_range=(1,5),)

In [10]:
vectorizer.fit(corpus)
tfidf.fit(corpus)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(1, 5), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [11]:
write_list_file('gram.txt', vectorizer.get_feature_names())
write_list_file("tfidf.txt", tfidf.get_feature_names())

In [13]:
def read_list_file(file):
    with open(file, "r") as f:
        data = f.readlines()
        return list(map(lambda x: x.strip(), data))

In [14]:
data = read_list_file('gram.txt')

In [15]:
# run through the list and remove the ones that occur in other ones
def remove_occurences(lst):
    i = 0
    lst = list(lst)
    while (len(lst)-1-i) > 0:
        current = lst[i] 
        nxt = lst[i+1]
        if current in nxt:
            lst.remove(current)
        else:
            i += 1
    return lst

In [16]:
cleaned_data = remove_occurences(data)

In [85]:
print("AVERAGE NUMBER OF INGREDIENTS PER RECIPE AFTER PROCESSING %f" % (len(cleaned_data)/NUM_EXAMPLES))

AVERAGE NUMBER OF INGREDIENTS PER RECIPE AFTER PROCESSING 3.268933


In [86]:
def term_counts(ingredients_set):
    counts = {}

    for string in ingredients_set:
        for part in string.split():
            if counts.get(part) is None:
                counts[part] = 0
            counts[part] += 1
    return counts
    

In [87]:
counts = term_counts(cleaned_data)
counts_series = pd.Series(counts)
counts_series.sort_values(ascending=False).head(1000).to_csv('counts.csv')

In [88]:
# opening a manually annotated file of non ingredient words using counts
with open('allrecipes_noningredient_words.txt', "r+") as f:
    annotated_data = list(map(lambda x: x.strip(), f.readlines()))

In [89]:
def remove_common_words(data_series):
    output = []
    for x in data_series:
        original_words = list(x.split())
        for word in x.split():
            if word in annotated_data or word.isdigit():
                original_words.remove(word)
        output.append(' '.join(original_words))
    return output

In [90]:
ingredients_set = set(remove_common_words(cleaned_data))

In [91]:
print("NUMBER OF INGREDIENTS NO HAMMING %d" % len(ingredients_set))

NUMBER OF INGREDIENTS NO HAMMING 36918


In [92]:
with open("ingredient_set.txt", "w+") as f:
    f.writelines(map(lambda x: "%s\n"%x,list(ingredients_set)))


In [93]:
ingreds = list(ingredients_set)

In [94]:
ingreds.sort()

In [95]:
# https://en.wikipedia.org/wiki/Hamming_distance distance between two strings
# a more relative hamming distance
def relative_hamming_score(string1, string2):
    dist_counter = 0
    max_val = min(len(string1),len(string2))
    for n in range(max_val):
        if string1[n] != string2[n]:
            dist_counter += 1
    if max_val is 0:
        return 0
    return dist_counter/max_val

In [96]:
THRESHOLD = 0.3
ingreds = list(ingredients_set)
ingreds.sort()
duplicates = []
for i in range(len(ingreds)-1):    
    if relative_hamming_score(ingreds[i], ingreds[i+1]) < THRESHOLD:
        print('matched')
        print(ingreds[i])
        print(ingreds[i+1])
        duplicates.append(ingreds[i]) # add first match to duplicates

matched

10x12
matched
10x12
12x12 inches each reynolds wrap
matched
12x12 squares aluminum
12x12 squares aluminum foil
matched
12x18 parchment
12x20 parchment
matched
18x24 inches reynolds wrap aluminum
18x24 inches reynolds wrap heavy
matched
1x1
1x1 matchsticks
matched
2x1
2x1 cubes
matched
2x2
2x2 sourdough
matched
2x2 sourdough
2x2 sourdough bread
matched
7up
7up cherry
matched
9x9 pan cornbread
9x9 pan cornbread cooled
matched
a1
a1 steak sauce
matched
about
about degrees degrees
matched
about diameter
about diameter red
matched
about diameter red
about diameter red bliss
matched
about each
about each down
matched
about inches
about inches long
matched
absinthe
absinthe based bitters
matched
acai berry pulp
acai berry sorbet
matched
acai pulp
acai pulp sambazon
matched
accents organic sprouted quinoa trio
accents organic sprouted rice trio
matched
achiote annatto seeds
achiote goya
matched
achiote goya
achiote powder
matched
acini di
acini di pepe
matched
acini di pepe
acini di p

baby potatoes quarters
matched
baby ray
baby red
matched
baby red
baby red gold
matched
baby red gold
baby red gold beets
matched
baby red gold beets
baby red gold potatoes
matched
baby red yellow
baby red yellow potatoes
matched
baby reds mashed potatoes
baby reds peas
matched
baby romaine lettuce
baby romaine lettuce mixed
matched
baby spinach
baby spinach bite
matched
baby spinach bite
baby spinach leaves
matched
baby spinach leaves
baby spinach leaves stems
matched
baby spinach leaves stems
baby spinach stems
matched
baby spinach stems
baby spinach taste
matched
baby squid rings
baby squid tubes
matched
baby squid tubes
baby squid tubes tentacles
matched
bacardi coconut
bacardi gold
matched
back rib at room temperature
back ribs
matched
back ribs
back ribs individual
matched
backbone heavy duty
backbone heavy duty scissors
matched
backfin crabmeat
backfin crabmeat picked
matched
backstrap
backstrap chunks
matched
bacon
bacon bacon bits
matched
bacon bits
bacon bits hormel
matched
b

bass
matched
bass
bass about
matched
bass fillets
bass fillets skinned
matched
basting mix
basting oil
matched
basting oil
basting sauce
matched
bay
bay blackened seasoning
matched
bay leaf
bay leaf crumbled
matched
bay leaves
bay leaves broken half
matched
bay scallops
bay scallops cleaned
matched
bay scallops cleaned
bay scallops cleaned patted
matched
bay scallops cleaned patted
bay scallops if
matched
bay scallops if
bay scallops raw
matched
bay scallops raw
bay scallops tough muscle
matched
bay seasoning
bay seasoning tm
matched
bay seasoning tm
bay seasoning tm taste
matched
bbq sauce
bbq spice rub parmesan
matched
beam
bean
matched
bean
bean bacon soup
matched
bean dip
bean dip fritos
matched
bean flour bob
bean flour needed
matched
bean mix
bean mix soup
matched
bean paste
bean paste kochujang
matched
bean soup
bean soup mix
matched
bean split
bean split scraped
matched
bean split scraped
bean split seeds
matched
bean split seeds
bean split seeds scraped
matched
bean sprouts
be

bitter melon flesh
matched
bitter melon flesh
bitter melon jelly
matched
bittermens fee
bittermens fee brothers
matched
bittermens fee brothers
bitters
matched
bitters
bitters angostura
matched
bittersweet chocolate
bittersweet chocolate chips
matched
bittersweet chocolate chips
bittersweet chocolate grated
matched
bittersweet chocolate grated
bittersweet chocolate melted
matched
bittersweet white chocolate
bittersweet white chocolate chips
matched
black
black angus
matched
black angus
black angus ribeye
matched
black angus ribeye
black angus ribeye steaks
matched
black bean garlic sauce
black bean sauce
matched
black bean sauce
black bean soup
matched
black bean soup
black bean vegetable blend
matched
black bean vegetable blend
black beans
matched
black beans
black beans bush
matched
black beans bush
black beans canned
matched
black beans canned
black beans drain
matched
black beans drain
black beans drain reserve
matched
black beans drain reserve
black beans green
matched
black beans

bread cubes plain croutons
matched
bread cubes walnut
bread cubes without crusts
matched
bread flour
bread flour all
matched
bread flour all
bread flour all purpose
matched
bread flour all purpose
bread flour half
matched
bread flour half
bread flour if
matched
bread flour if
bread flour king
matched
bread flour king
bread flour needed
matched
bread flour needed
bread flour plus
matched
bread french baguette
bread french bread
matched
bread inches diameter
bread inches long
matched
bread loaf hollowed
bread loaf hollowed make
matched
bread machine
bread machine yeast
matched
bread machine yeast
bread mix
matched
bread mix
bread mix jiffy
matched
bread processed crumbs
bread processed fine crumbs
matched
bread roll mix yeast
bread rolls
matched
bread rolls
bread rolls split half
matched
bread rounds
bread rounds brown
matched
bread rounds brown
bread rounds half
matched
bread rounds half
bread rounds triangles
matched
bread split
bread split half
matched
bread stuffing
bread stuffing cu

...

sugar sugar raw
matched
sugar sugar raw
sugar sugar substitute honey
matched
sugar vanilla extract
sugars
matched
sukiyaki ingredients
sukiyaki sauce
matched
sumac
sumac powder
matched
summer savory
summer savory chubritsa
matched
summer squash
summer squash chunks
matched
summer squash zucchini
summer squashes
matched
sun
sun gold
matched
sun tomato
sun tomato halves
matched
sun tomato halves
sun tomato oil
matched
sun tomato oil
sun tomato oregano
matched
sun tomato paste
sun tomato pesto
matched
sun tomato spread
sun tomatoes
matched
sun tomatoes
sun tomatoes marinated
matched
sun tomatoes marinated
sun tomatoes not
matched
sun tomatoes not
sun tomatoes not oil
matched
sun tomatoes not oil
sun tomatoes oil
matched
sun tomatoes oil
sun tomatoes rehydrated
matched
sundried tomato mayonnaise see
sundried tomato mayonnaise see below
matched
sundried tomato mayonnaise see below
sundried tomatoes
matched
sunflower kernels
sunflower meal
matched
sunflower meal
sunflower oil
matched
sunflow

tart apple
tart apple cored
matched
tart apple cored
tart apple fugi
matched
tart apple fugi
tart apple granny
matched
tart apple granny
tart apples
matched
tart apples
tart apples cored
matched
tart cherries
tart cherries door
matched
tart cooking apples
tart cooking apples cored
matched
tart green apple
tart green apples
matched
tart green apples
tart green apples cored
matched
tart red
tart red apple
matched
tart shells
tart shells baked
matched
tartar
tartar sauce
matched
tartaric acid
tarts
matched
tasso ham
tasso ham matchsticks
matched
taste
taste broken
matched
taste flaked
taste grated
matched
taste leaves
taste leaves stems
matched
taste plus extra
taste plus extra dusting
matched
taste see ingredient
taste see ingredient note
matched
taste see ingredient note
taste serving
matched
taste thai
taste thai peanut
matched
taste thai peanut
taste thai peanut satay
matched
taste wear
taste wear gloves
matched
tater
tater tots
matched
tea
tea bags
matched
tea bags
tea bags black tea

turkey meat
turkey meat light
matched
turkey meat light
turkey meat light dark
matched
turkey meat pulled
turkey meat taste
matched
turkey meatballs
turkey neck
matched
turkey neck
turkey neck giblets
matched
turkey neck giblets
turkey neck giblets reserved
matched
turkey neck heart gizzard
turkey neck stock
matched
turkey neck stock
turkey necks
matched
turkey pan drippings
turkey pan drippings fat
matched
turkey pan drippings fat
turkey pan drippings fat skimmed
matched
turkey pan drippings fat skimmed
turkey pan drippings turkey
matched
turkey pepperoni
turkey pepperoni hormel
matched
turkey plain
turkey plain smoked
matched
turkey sausage
turkey sausage bite
matched
turkey sausage bite
turkey sausage casings
matched
turkey sausage casings
turkey sausage casings discarded
matched
turkey sausage casings discarded
turkey sausage chunks
matched
turkey sausage chunks
turkey sausage crumbles
matched
turkey sausage link without casing
turkey sausage links
matched
turkey sausage links
turk

vinaigrette salad dressing taste
vinaigrette taste
matched
vine ripe tomato
vine ripe tomatoes
matched
vine ripened tomato cored
vine ripened tomatoes
matched
vinegar
vinegar acidity
matched
vinegar based carolina
vinegar based carolina bbq
matched
vinegar based hot pepper
vinegar based hot pepper sauce
matched
vinegar plus
vinegar potato chips
matched
vinegar rice vinegar
vinegar rice wine vinegar
matched
vinegar taste
vinegar white wine vinegar
matched
virgin coconut oil
virgin coconut oil melted
matched
virgin coconut oil melted
virgin coconut oil room
matched
virgin coconut oil room
virgin coconut oil room temperature
matched
virgin olive oil
virgin olive oil aged balsamic
matched
virgin olive oil brushing
virgin olive oil canola
matched
virgin olive oil canola
virgin olive oil drizzling
matched
virgin olive oil drizzling
virgin olive oil pesto
matched
virgin olive oil pesto
virgin olive oil plus
matched
vital wheat gluten
vital wheat gluten flour
matched
vitar liquid
vitar liquid 

matched
without casing
without casings
matched
without flavor lightly
without oil
matched
without oil
without peanuts
matched
without peanuts
without peel
matched
without peel
without pulp
matched
without shells
without skin
matched
without skin
without tails
matched
without tails
without tails deveined
matched
won
won ton
matched
won ton
won ton wrappers
matched
wonton
wonton filling
matched
wonton wrappers
wonton wrappers needed
matched
wonton wrappers needed
wontons
matched
wood chips
wood chips needed
matched
wooden skewers
wooden skewers coffee
matched
wooden skewers coffee
wooden skewers coffee stirrers
matched
wooden skewers coffee stirrers
wooden skewers long
matched
wooden skewers long
wooden skewers needed
matched
wooden skewers needed
wooden skewers water
matched
wooden skewers water
wooden stake
matched
wooden stake
wooden sticks
matched
worcestershire
worcestershire sauce
matched
worcestershire sauce
worcestershire sauce lea
matched
worcestershire sauce lea
worcestershire 

In [97]:
cleaned_ingredients_set = list(ingredients_set)
for dupe in duplicates:
    cleaned_ingredients_set.remove(dupe)
cleaned_ingredients_set.sort()
print("NUM OF INGREDIENTS AFTER HAMMING %d" % len(cleaned_ingredients_set))

NUM OF INGREDIENTS AFTER HAMMING 19147


In [98]:
write_list_file('ingredients_set.txt', cleaned_ingredients_set)

In [99]:
counts = term_counts(cleaned_ingredients_set)
counts_series = pd.Series(counts)
counts_series.sort_values(ascending=False).head(1000).to_csv('counts.csv')

In [100]:
len(cleaned_ingredients_set)

19147

In [101]:
cleaned_ingredients_set

['12x12 inches each reynolds wrap',
 '12x12 squares aluminum foil',
 '12x16 parchment paper',
 '12x16 squares aluminum',
 '12x18 inches each reynolds wrap',
 '12x18 nylon',
 '12x20 parchment',
 '12x36 inches reynolds wrap non',
 '18x15 inches each reynolds wrap',
 '18x24 inches reynolds wrap heavy',
 '1x1 matchsticks',
 '24x8x1 untreated cedar plank',
 '2x1 cubes',
 '2x2 sourdough bread',
 '2x3',
 '3x6',
 '4x4',
 '60l grain bag',
 '6x1',
 '6x9 sheet puff',
 '7up cherry',
 '7up needed',
 '7up taste',
 '8x2 inches',
 '9x9 pan cornbread cooled',
 'a1 steak sauce',
 'a1 taste',
 'abalone without shell',
 'about degrees degrees',
 'about diameter red bliss',
 'about each down',
 'about inches long',
 'about lime',
 'about sea salt',
 'about thickness',
 'above',
 'absinthe based bitters',
 'absinthe substitute',
 'absolut citron',
 'absolut ruby red',
 'abuelita',
 'ac cent',
 'acai berry sorbet',
 'acai juice',
 'acai pulp sambazon',
 'accent flavor enhancer',
 'accents organic sprouted ri